#### The test_pid_model ipnb file calculates the accuracy of the model bounding box and label prediction using Mean square error and percentage based accuracy.

In [8]:
import io
import os
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow

from google.oauth2 import service_account

class_area = {'21': 1985, '2': 5415, '18': 6969, '26': 17849, '8': 7037, '20': 6110, '25': 6532, '32': 16411, '28': 17179, '7': 6771, '9': 6689, '12': 5510, '22': 5795, '24': 2808, '11': 6129, '23': 8645, '29': 17181, '4': 6132, '31': 16001, '5': 4808, '15': 3737, '17': 5721, '19': 6769, '30': 11940, '16': 3564, '13': 6558, '1': 4575, '6': 3126, '10': 8179, '27': 18761, '14': 13607, '3': 6903}

TEST_DATASET_PATH = "D:/Veilex/_test/train-segment-data/datasets/custom_dataset/dataset/test"
FULL_TEST_DATASET_PATH="D:/Veilex/_test/dataset/DigitizePID_Dataset"
images_dir = "images"
labels_dir = "labels"

# the file id for the model in google dirve
MODEL_FILE_ID = "1rd0GmerO8wzWBcAr5e38_WOXYLJqTDKl"
model_file_id = MODEL_FILE_ID
model_file_name = "yolo-model-pid.pt"

In [ ]:
SCOPES = ["https://www.googleapis.com/auth/drive"]

CREDENTIAL_FILE_NAME = "./credentials.json"
TOKEN_FILE_NAME = "./token.json"

def download_file(file_id, credential_file_name: str):
    file = None
    creds = None

    if os.path.exists(TOKEN_FILE_NAME):
        creds = Credentials.from_authorized_user_file(TOKEN_FILE_NAME, SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                credential_file_name, SCOPES
            )
            creds = flow.run_local_server(port=0)
        with open(TOKEN_FILE_NAME, "w") as token:
            token.write(creds.to_json())

    try:
        service = build("drive", "v3", credentials=creds)

        request = service.files().get_media(fileId=file_id)
        file = io.BytesIO()
        downloader = MediaIoBaseDownload(file, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()
            print(f"Download {int(status.progress() * 100)}.")

    except HttpError as error:
        print(f"An error occurred: {error}")
        file = None

    return file.getvalue()

def upload_yolo_file(model_file_id: str):
    yolo_model_pid_bytes = download_file(file_id=model_file_id, credential_file_name=CREDENTIAL_FILE_NAME)
    if not os.path.exists(f"./{model_file_name}"):
        with open(f"./{model_file_name}", 'wb') as f:
            f.write(yolo_model_pid_bytes)

    return f"./{model_file_name}"

# file_path = upload_yolo_file(model_file_id)
file_path = download_file_with_service_account(MODEL_FILE_ID, "./service-account-key.json")
model_file_path = file_path if file_path is not None else f"./{model_file_name}"

In [ ]:
from ultralytics import YOLO


model = YOLO(model_file_path)

In [ ]:
from typing import List, Tuple
from PIL import Image

from src.models import Vertex, PidDataPoint, Symbol
from src.utils import denormalize_coordinates

# this is a YOLO formatted based dataset that has to be extracted from the directory.
def read_yolo_label_data(label_path: str) -> List[Tuple[str, float, float, float, float]]:
    with open(label_path, 'r') as file:
        lines = file.readlines()

    labels = []
    for line in lines:
        class_id, x, y, _x, _y = line.strip().split()
        labels.append((class_id, float(x), float(y), float(_x), float(_y)))
    return labels
    

pid_test_dataset = []

dataset_images_path = f"{TEST_DATASET_PATH}/{images_dir}"
dataset_labels_path = f"{TEST_DATASET_PATH}/{labels_dir}"
image_files = [os.path.join(dataset_images_path, f) for f in os.listdir(dataset_images_path) if os.path.isfile(os.path.join(dataset_images_path, f))]

for image_file_path in image_files:
    datapoint = PidDataPoint(
        image_path=image_file_path
    )
    image = Image.open(image_file_path)
    image_file_name = '.'.join(image_file_path.split("\\")[-1].split(".")[0:-1])
    label_dp_path = os.path.join(dataset_labels_path, f"{image_file_name}.txt")
    if os.path.isfile(label_dp_path):
        yolo_datalabel = read_yolo_label_data(label_dp_path)
        symbols = []
        for l in yolo_datalabel:
            class_id, x, y, _x, _y = l
            x, y, _x, _y = denormalize_coordinates([x, y, _x, _y], image.width, image.height)
            symbols.append(
                Symbol(
                    label=class_id,
                    name="",
                    pointSrc=Vertex(x=x, y=y),
                    pointDest=Vertex(x=_x, y=_y)
                )
            )
        datapoint.symbols = symbols
    
    pid_test_dataset.append(datapoint)


In [ ]:
from src.services import PredictSymbolsService


def predict_results():
    predicted_results = []
    for index, d in enumerate(pid_test_dataset[:]):
        predict_service = PredictSymbolsService(
            image_path=d.image_path,
            model_path=model_file_path
        )

        bboxes_with_labels = predict_service.predict_bounding_boxes()
        predicted_results.append(bboxes_with_labels)

    return predicted_results

predicted_results: List[List[Tuple[List[int], str]]] = predict_results()

In [ ]:
# remove all the symbols that have been detected but were cutted of by segmenting the images.
def filter_out_cutoff_symbols(symbols: list[list[Tuple[list[int, int, int, int], str]]]):
    symbols_filtered = []
    for symbol in symbols:
        filtered = []
        for coordinates in symbol:
            x, y, _x, _y = coordinates[0]
            label = coordinates[1]
            area = abs(_x - x) * abs(_y - y)
            if(area >= class_area[label] * 0.7):
                filtered.append(coordinates)
        symbols_filtered.append(filtered)

    return symbols_filtered

predicted_results = filter_out_cutoff_symbols(
    predicted_results
)


In [ ]:
from src.models.bounding_box import BoundingBox
from src.services.image_display_serivce import ImageDisplayService
from src.utils import convert_raw_data_to_bounding_box

def dispaly_pid_image(index: int):
    display_service = ImageDisplayService(
        image_path=pid_test_dataset[index].image_path,
        bounding_boxes= [
            BoundingBox(
                name=n[1],
                pointSrc=Vertex(x=int(n[0][0]), y=int(n[0][1])),
                pointDest=Vertex(x=int(n[0][2]), y=int(n[0][3]))
            )
            for n in predicted_results[index]
        ]
    )
    display_service.display_image_with_bbox()

dispaly_pid_image(87)

In [ ]:
import numpy as np
from scipy.optimize import linear_sum_assignment

# mean square error calculation w.r.t the bounding boxes.

def mean_square_error_with_matching(actual_boxes, predicted_boxes) -> float:
    """
    Calculate the mean square error (MSE) between actual and predicted bounding boxes,
    accounting for mismatched order by using optimal matching (Hungarian algorithm).
    
    Parameters:
    actual_boxes: List of actual bounding boxes [(x1, y1, x2, y2), ...]
    predicted_boxes: List of predicted bounding boxes [(x1, y1, x2, y2), ...]

    Returns:
    float: Mean square error value
    """

    if len(actual_boxes) > len(predicted_boxes):
        for _ in range(abs(len(actual_boxes) - len(predicted_boxes))):
            predicted_boxes.append([0, 0, 0, 0])
    
    if len(predicted_boxes) > len(actual_boxes):
        for _ in range(abs(len(actual_boxes) - len(predicted_boxes))):
            actual_boxes.append([0, 0, 0, 0])

    if len(actual_boxes) != len(predicted_boxes):
        raise ValueError("The number of actual and predicted boxes must be the same.")
    
    actual_boxes = np.array(actual_boxes)
    predicted_boxes = np.array(predicted_boxes)
    
    # Compute the cost matrix (pairwise squared errors)
    cost_matrix = np.zeros((len(actual_boxes), len(predicted_boxes)))
    for i, actual in enumerate(actual_boxes):
        for j, predicted in enumerate(predicted_boxes):
            cost_matrix[i, j] = np.mean((actual - predicted) ** 2)
    
    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    
    mse = 0.0
    for i, j in zip(row_ind, col_ind):
        mse += cost_matrix[i, j]
    
    mse /= len(actual_boxes)
    return mse

def bbox_area_valid(annotation_prediction):
    bbox = annotation_prediction[0]
    class_id = annotation_prediction[1]
    predicted_area = (abs(bbox[0] - bbox[2])) * (abs(bbox[1] - bbox[3]))
    return predicted_area > class_area[class_id] * 0.6

# mean squre error in testing the images with symbols
def calculate_test_pid_mse():
    no_of_test_data = len(predicted_results)
    not_including_in_testing = 0
    sum = 0
    for index, d in enumerate(predicted_results):
        # filtered_result = [bl for bl in d if bbox_area_valid(bl)] # [bl[0] for bl in d]
        predicted_bboxes = [bl[0] for bl in d] # [bl[0] for bl in filtered_result] 
        actual_bboxes = []
        for s in pid_test_dataset[index].symbols:
            x, y = s.pointSrc.get_dimensions()
            _x, _y = s.pointDest.get_dimensions()
            actual_bboxes.append([x, y, _x, _y])
        
        mse = 0
        try:
            mse += mean_square_error_with_matching(actual_bboxes, predicted_bboxes)
        except:
            not_including_in_testing += 1
            print(f"could not caculate MSE of the datapoint {index}") 

        sum += mse
    print("not included in dataset ", not_including_in_testing)
    print("Mean Square Error, ", sum / (no_of_test_data - not_including_in_testing))

calculate_test_pid_mse()

In [ ]:
import numpy as np
from scipy.optimize import linear_sum_assignment


# accuracy of dataset w.r.t the bounding boxes.

def calculate_iou(box1, box2) -> float:
    """
    Calculate Intersection over Union (IoU) between two bounding boxes.
    box1, box2: (x1, y1, x2, y2)
    """
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    
    inter_area = max(0, x2 - x1) * max(0, y2 - y1)
    
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    
    union_area = box1_area + box2_area - inter_area
    
    return inter_area / union_area if union_area != 0 else 0

def accuracy_with_iou(actual_boxes, predicted_boxes, iou_threshold=0.5) -> float:
    """
    Calculate the percentage accuracy of the predicted bounding boxes using IoU.
    
    Parameters:
    actual_boxes: List of actual bounding boxes [(x1, y1, x2, y2), ...]
    predicted_boxes: List of predicted bounding boxes [(x1, y1, x2, y2), ...]
    iou_threshold: IoU threshold to consider a match as correct
    
    Returns:
    float: Accuracy percentage
    """
    if len(actual_boxes) > len(predicted_boxes):
        for _ in range(abs(len(actual_boxes) - len(predicted_boxes))):
            predicted_boxes.append([0, 0, 0, 0])
    
    if len(predicted_boxes) > len(actual_boxes):
        for _ in range(abs(len(actual_boxes) - len(predicted_boxes))):
            actual_boxes.append([0, 0, 0, 0])

    if len(actual_boxes) != len(predicted_boxes):
        raise ValueError("The number of actual and predicted boxes must be the same.")
    
    # Compute the IoU cost matrix
    cost_matrix = np.zeros((len(actual_boxes), len(predicted_boxes)))
    for i, actual in enumerate(actual_boxes):
        for j, predicted in enumerate(predicted_boxes):
            cost_matrix[i, j] = -calculate_iou(actual, predicted)  # Negate for maximization
    
    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    
    correct_matches = 0
    for i, j in zip(row_ind, col_ind):
        iou = -cost_matrix[i, j]  # Undo the negation
        if iou >= iou_threshold:
            correct_matches += 1
    
    # Compute accuracy
    accuracy = (correct_matches / len(actual_boxes)) * 100
    return accuracy

# percentage accuracy of PID
def calculate_test_pid_accuracy():
    no_of_test_dataset = len(predicted_results)
    not_including_in_testing = 0
    sum = 0
    for index, d in enumerate(predicted_results):
        predicted_bboxes = [bl[0] for bl in d]

        actual_bboxes = []
        for s in pid_test_dataset[index].symbols:
            x, y = s.pointSrc.get_dimensions()
            _x, _y = s.pointDest.get_dimensions()
            actual_bboxes.append([x, y, _x, _y])
        
        accuracy = 0
        try:
            accuracy += accuracy_with_iou(actual_bboxes, predicted_bboxes)
        except:
            not_including_in_testing += 1
            print(f"Could not test MSE of the datapoint {index}") 

        sum += accuracy

    print("Accuracy")
    print(sum / (no_of_test_dataset - not_including_in_testing))
    print("datapoints not included in testing ", not_including_in_testing)

calculate_test_pid_accuracy()

In [ ]:
# calculation of label accuracy with the segmented dataset.

def calculate_label_accuracy(predicted_result_set, actual_dataset):
    sum_accuracy = 0
    pid_test_dataset = actual_dataset
    for index, d in enumerate(predicted_result_set):
        predicted_labels = [x[1] for x in d]
        actual_symbol_labels = [s.label for s in pid_test_dataset[index].symbols]
        s_dict = {}
        for s in actual_symbol_labels:
            if(s not in s_dict):
                s_dict[s] = 1
            else:
                s_dict[s] += 1

        for s in predicted_labels:
            if(s in s_dict):
                s_dict[s] -= 1

        incorrect_result_sum = sum([abs(val) for val in s_dict.values()])

        sum_accuracy += ((len(actual_symbol_labels) - incorrect_result_sum) / len(actual_symbol_labels))

    return sum_accuracy / len(predicted_result_set)

### Testing the accuracy on full images

In [ ]:
pid_dataset_path = "D:/Veilex/_test/dataset/DigitizePID_Dataset"
from config import config
from src.services import DataConverterService, PredictSymbolsService

# change the config here. This should not be done anywhere else btw, this is exception
config.dataset_path = pid_dataset_path
config.image_dir_name = "image_2"
config.annotation_dir_name = "annotations"

data_service = DataConverterService()
full_pid_dataset = data_service.load_dataset()

def predict_full_dataset_results():
    predicted_results = []
    for d in full_pid_dataset[0:20]:
        predict_service = PredictSymbolsService(
            image_path=d.image_path,
            model_path=model_file_path
        )

        bboxes_with_labels = predict_service.predict_bounding_boxes(shifting=True)
        predicted_results.append(bboxes_with_labels)

    return predicted_results

predicted_full_results: List[PidDataPoint] = predict_full_dataset_results()
    

In [ ]:
from src.utils import convert_points_to_bounding_box
from src.services import ImageDisplayService

index_to_select = 9

instance = predicted_full_results[index_to_select]
service = ImageDisplayService(
    image_path=full_pid_dataset[index_to_select].image_path,
    bounding_boxes=[convert_points_to_bounding_box(i[0]) for i in instance ]
)

service.display_image_with_bbox()

In [ ]:
def calculate_full_pid_dataset_accuracy():
    not_including_in_testing = 0
    sum = 0
    for index, d in enumerate(predicted_full_results):
        predicted_bboxes = [bl[0] for bl in d]

        actual_bboxes = []
        for s in full_pid_dataset[index].symbols:
            x, y = s.pointSrc.get_dimensions()
            _x, _y = s.pointDest.get_dimensions()
            actual_bboxes.append([x, y, _x, _y])
        
        accuracy = 0
        try:
            print( len(actual_bboxes), len(predicted_bboxes) )
            accuracy += accuracy_with_iou(actual_bboxes, predicted_bboxes)
        except:
            not_including_in_testing += 1
            print(f"Could not test MSE of the datapoint {index}") 

        sum += accuracy

    print("Accuracy")
    print(sum / (len(predicted_full_results) - not_including_in_testing))
    print("datapoints not included in testing ", not_including_in_testing)

calculate_full_pid_dataset_accuracy()

In [ ]:
calculate_label_accuracy(predicted_full_results, full_pid_dataset[0:20])